In [ ]:
#THIS APPROACH DID NOT WORK 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import lightgbm as lgb


In [10]:
train = pd.read_csv("MiNDAT.csv")
test = pd.read_csv("SPECIMEN.csv")


In [11]:
train = train.drop_duplicates()

In [25]:
target_col = "CORRUCYSTIC_DENSITY"
id_col = "LOCAL_IDENTIFIER"
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor

# Example: define categorical and numeric columns
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
numeric_cols = [col for col in X_train.columns if X_train[col].dtype != "object"]

# Preprocessor with safe OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_cols)
    ]
)

# Full pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42))
])

# Train again
pipeline.fit(X_train, y_train)

# Validation
y_pred = pipeline.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
print("✅ Validation RMSE:", rmse)

y = train[target_col]
X = train.drop(columns=[target_col, id_col])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003378 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11011
[LightGBM] [Info] Number of data points in the train set: 8820, number of used features: 66
[LightGBM] [Info] Start training from score 448.512124


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


TypeError: got an unexpected keyword argument 'squared'

In [13]:
mask = ~y.isna()
X = X.loc[mask]
y = y.loc[mask]


In [14]:
for col in X.columns:
    if X[col].dtype in ["float64", "int64"]:
        X[col] = X[col].fillna(X[col].median())
    else:
        X[col] = X[col].fillna(X[col].mode()[0])

for col in test.columns:
    if col != id_col:
        if test[col].dtype in ["float64", "int64"]:
            test[col] = test[col].fillna(test[col].median())
        else:
            test[col] = test[col].fillna(test[col].mode()[0])

In [15]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns


In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [18]:
model = lgb.LGBMRegressor(
    num_leaves=50,
    max_depth=-1,
    learning_rate=0.05,
    n_estimators=1000,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])


In [20]:
from sklearn.metrics import mean_squared_error
import numpy as np

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)

# compute RMSE manually
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)

print("✅ Validation RMSE:", rmse)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002648 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11011
[LightGBM] [Info] Number of data points in the train set: 8820, number of used features: 66
[LightGBM] [Info] Start training from score 448.512124
✅ Validation RMSE: 170.34788802711432


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [21]:
pipeline.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005914 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11011
[LightGBM] [Info] Number of data points in the train set: 11026, number of used features: 66
[LightGBM] [Info] Start training from score 449.481411


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [24]:
# Get feature names used during training
train_features = pipeline.named_steps['preprocessor'].feature_names_in_

# Drop ID column from test
X_test = test.drop(columns=[id_col])

# Check for missing and extra columns
missing_cols = set(train_features) - set(X_test.columns)
extra_cols   = set(X_test.columns) - set(train_features)

print("🔍 Column Check")
print("Missing in test set:", missing_cols)
print("Extra in test set:", extra_cols)

# Align columns by reindexing test to match training
X_test = X_test.reindex(columns=train_features, fill_value=0)

# Predict
preds = pipeline.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    id_col: test[id_col],
    target_col: preds
})

submission.to_csv("submission2.csv", index=False)
print("✅ submission2.csv created successfully!")


🔍 Column Check
Missing in test set: {'b1oRb13', '.b6nl', '.6AvGp', "F'3Ku", 'v0rt3X', 'v1rt3X', 'r1Ng', 'TSWm', 'T\\!', '<!!', 'Jv[i', 'Z~x0<k', 'fPqsI', 'ZZw3=!t', '9Z/5)2', 'U"r', '+U@', 'Kj,', ';<"<i(T', '!;@Jw', 'maT_r', 'A>.', '&%)LTaWRb', 'MINDSPIKE_VERSION', 'ZVf', '@wnsk>R', "0HU2N='U", '~7*', 'b2oRb13', '.o<m', ']xq', 'r2Ng', 'i]7V', '3I\\y', '14W$Q', '^%a;', 'hp!', 'PZ8', 'ZrK', 'jNhEum', '>?64:', '|G}', '%IiL7w', '@V9', "w-u:jN'qI", 'vzo."'}
Extra in test set: {'CORRUCYSTIC_DENSITY'}


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''